In [1]:
import pandas as pd
import sqlite3

In [2]:
# Charger le CSV
df = pd.read_csv("data/24-25.csv")

# Créer une base SQLite (ici en mémoire)
conn = sqlite3.connect(":memory:")

# Importer le DataFrame dans SQLite comme une table appelée "L1"
df.to_sql("L1", conn, index=False, if_exists="replace")

306

In [3]:
print(df.columns.tolist())

['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'BFH', 'BFD', 'BFA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', '1XBH', '1XBD', '1XBA', 'MaxH', 'MaxD', 'MaxA', 'AvgH', 'AvgD', 'AvgA', 'BFEH', 'BFED', 'BFEA', 'B365>2.5', 'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'BFE>2.5', 'BFE<2.5', 'AHh', 'B365AHH', 'B365AHA', 'PAHH', 'PAHA', 'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA', 'BFEAHH', 'BFEAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'BFCH', 'BFCD', 'BFCA', 'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', '1XBCH', '1XBCD', '1XBCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD', 'AvgCA', 'BFECH', 'BFECD', 'BFECA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'BFEC>2.5', 'BFEC<2.5', 'AHCh', 'B365CAHH', 'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH',

# 1. Analyses sportives "classiques"

## Classement des équipes

- total de points (3 victoire, 1 nul, 0 défaite), différence de buts (FTHG - FTAG), trié comme un vrai classement de Ligue 1

In [4]:
query = """
SELECT Team, SUM(BP) AS BP, SUM(BC) AS BC, SUM(BP) - SUM(BC) AS DB, SUM(Points) AS Points
FROM (
        -- Equipe Domicile
        SELECT HomeTeam AS Team, FTHG AS BP, FTAG AS BC,
        CASE
            WHEN FTR = 'H' THEN 3
            WHEN FTR = 'A' THEN 0
            WHEN FTR = 'D' THEN 1
        END AS Points
        FROM L1

        UNION ALL

        -- Equipe Extérieur
        SELECT AwayTeam AS Team, FTAG AS BP, FTHG AS BC,
        CASE
            WHEN FTR = 'H' THEN 0
            WHEN FTR = 'A' THEN 3
            WHEN FTR = 'D' THEN 1
        END AS Points
        FROM L1

) AS Matchs
GROUP BY Team
ORDER BY Points DESC, DB DESC
"""
result = pd.read_sql(query, conn)
print(result)

           Team  BP  BC  DB  Points
0      Paris SG  92  35  57      84
1     Marseille  74  47  27      65
2        Monaco  63  41  22      61
3          Nice  66  41  25      60
4         Lille  52  36  16      60
5          Lyon  65  46  19      57
6    Strasbourg  56  44  12      57
7          Lens  42  39   3      52
8         Brest  52  59  -7      50
9      Toulouse  44  43   1      42
10      Auxerre  48  51  -3      42
11       Rennes  51  50   1      41
12       Nantes  39  52 -13      36
13       Angers  32  53 -21      36
14     Le Havre  40  71 -31      34
15        Reims  33  47 -14      33
16   St Etienne  39  77 -38      30
17  Montpellier  23  79 -56      16


## Performances domicile / extérieur

- comparer points, buts marqués et encaissés à domicile vs à l’extérieur

In [6]:
query = """
SELECT H.Team, H.BPHome, H.BCHome, H.DBHome, H.HomePoints,
        A.BPAway, A.BCAway, A.DBAway, A.AwayPoints
FROM (
        SELECT HomeTeam AS Team, SUM(FTHG) AS BPHome, SUM(FTAG) AS BCHome, SUM(FTHG) - SUM(FTAG) AS DBHome,
        SUM(CASE
            WHEN FTR = 'H' THEN 3
            WHEN FTR = 'A' THEN 0
            WHEN FTR = 'D' THEN 1
        END) AS HomePoints
        FROM L1
        GROUP BY Team
        ORDER BY HomePoints DESC
) AS H
JOIN (
        SELECT AwayTeam AS Team, SUM(FTAG) AS BPAway, SUM(FTHG) AS BCAway, SUM(FTAG) - SUM(FTHG) AS DBAway,
        SUM(CASE
            WHEN FTR = 'H' THEN 0
            WHEN FTR = 'A' THEN 3
            WHEN FTR = 'D' THEN 1
        END) AS AwayPoints
        FROM L1
        GROUP BY Team
        ORDER BY AwayPoints DESC
) AS A
ON H.Team = A.Team
ORDER BY (H.HomePoints + A.AwayPoints) DESC
"""
result = pd.read_sql(query, conn)
print(result)

           Team  BPHome  BCHome  DBHome  HomePoints  BPAway  BCAway  DBAway  \
0      Paris SG      45      16      29          44      47      19      28   
1     Marseille      41      23      18          34      33      24       9   
2        Monaco      38      16      22          39      25      25       0   
3          Nice      38      15      23          37      28      26       2   
4         Lille      31      18      13          37      21      18       3   
5    Strasbourg      33      20      13          35      23      24      -1   
6          Lyon      37      21      16          35      28      25       3   
7          Lens      19      22      -3          23      23      17       6   
8         Brest      31      21      10          33      21      38     -17   
9       Auxerre      24      17       7          27      24      34     -10   
10     Toulouse      20      22      -2          20      24      21       3   
11       Rennes      25      17       8          28 

## Buts par période

- comparer HTHG / HTAG (mi-temps) vs FTHG / FTAG (fin de match) → voir quelles équipes marquent plutôt en 2ème mi-temps

In [19]:
query = """
SELECT Team, SUM(FirstHalf) AS FirstHalf, SUM(SecondHalf) AS SecondHalf, Count(*) AS Matchs
FROM (
        SELECT HomeTeam AS Team, HTHG AS FirstHalf, (FTHG - HTHG) AS SecondHalf
        FROM L1

        UNION ALL

        SELECT AwayTeam AS Team, HTAG AS FirstHalf, (FTAG - HTAG) AS SecondHalf
        FROM L1
)
GROUP BY Team
Order BY SecondHalf DESC
"""
result = pd.read_sql(query, conn)
print(result)

           Team  FirstHalf  SecondHalf  Matchs
0      Paris SG         37          55      34
1     Marseille         26          48      34
2          Lyon         22          43      34
3        Monaco         23          40      34
4    Strasbourg         23          33      34
5        Rennes         23          28      34
6          Nice         38          28      34
7      Toulouse         17          27      34
8         Lille         25          27      34
9         Brest         29          23      34
10      Auxerre         25          23      34
11   St Etienne         17          22      34
12         Lens         20          22      34
13     Le Havre         18          22      34
14       Nantes         19          20      34
15       Angers         14          18      34
16        Reims         17          16      34
17  Montpellier          8          15      34


## Attaque et défense

- équipe qui tire le plus (HS + AS), équipe qui cadre le plus (HST + AST), équipe qui encaisse le plus de cartons (HY, AY, HR, AR)

In [45]:
query = """
SELECT Team, SUM(TotalShots) AS TotalShots, SUM(ShotsOnTarget) AS ShotsOnTarget, SUM(YellowCards) AS YellowCards, SUM(RedCards) AS RedCards
FROM (
        SELECT HomeTeam AS Team,
               HS AS TotalShots,
               HST AS ShotsOnTarget,
               HY AS YellowCards,
               HR AS RedCards
        FROM L1

        UNION ALL

        SELECT AwayTeam AS Team,
               "AS" AS TotalShots,
               AST AS ShotsOnTarget,
               AY AS YellowCards,
               AR AS RedCards
        FROM L1
) AS Stats
GROUP BY Team
ORDER BY SUM(ShotsOnTarget) DESC
"""
result = pd.read_sql(query, conn)
print(result)

           Team  TotalShots  ShotsOnTarget  YellowCards  RedCards
0      Paris SG         627            282           37         0
1     Marseille         476            205           64         6
2          Nice         489            195           59         5
3        Monaco         491            189           57         4
4          Lyon         443            187           56         0
5         Lille         435            166           75         2
6        Rennes         431            164           67         2
7          Lens         492            162           86         8
8         Brest         411            159           66         3
9      Toulouse         399            156           54         3
10      Auxerre         377            152           67         4
11   Strasbourg         353            144           70         3
12       Nantes         346            128           69         4
13   St Etienne         362            121           65         3
14  Montpe

# 2. Analyses "data métier"

## Corrélation tirs / buts

- moyenne de tirs cadrés vs buts marqués, pour identifier les équipes les plus efficaces

In [66]:
query = """
SELECT Team,
       SUM(ShotsOnTarget) AS TotalShotsOnTarget,
       SUM(ShotsOnTarget) / COUNT(*) AS AvgShotsOnTargetPerMatch,
       SUM(Goals) AS TotalGoals,
       SUM(Goals) / COUNT(*) AS AvgGoalsPerMatch,
       SUM(Goals) * 1.0 / NULLIF(SUM(ShotsOnTarget), 0) AS Efficiency,
       COUNT(*) AS Matchs
FROM (
        SELECT HomeTeam AS Team, FTHG AS Goals, HST AS ShotsOnTarget
        FROM L1

        UNION ALL

        SELECT AwayTeam AS Team, FTAG AS Goals, AST AS ShotsOnTarget
        FROM L1
)
GROUP BY Team
ORDER BY Efficiency DESC
"""
result = pd.read_sql(query, conn)
print(result)

           Team  TotalShotsOnTarget  AvgShotsOnTargetPerMatch  TotalGoals  \
0    Strasbourg                 144                         4          56   
1     Marseille                 205                         6          74   
2      Le Havre                 113                         3          40   
3          Lyon                 187                         5          65   
4          Nice                 195                         5          66   
5        Monaco                 189                         5          63   
6        Angers                  96                         2          32   
7         Brest                 159                         4          52   
8      Paris SG                 282                         8          92   
9    St Etienne                 121                         3          39   
10      Auxerre                 152                         4          48   
11        Lille                 166                         4          52   

## Fair-play / agressivité

- ratio fautes (HF, AF) et cartons (HY, AY) par match → voir si une équipe est plus "rugueuse"

In [80]:
query = """
SELECT Team,
       SUM(YellowCards) + SUM(RedCards) AS TotalCards,
       SUM(YellowCards) AS TotalYellowCards,
       SUM(RedCards) AS TotalRedCards,
       (SUM(YellowCards) + SUM(RedCards)) / COUNT(*) AS CardsPerMatch,
       SUM(Fouls) AS TotalFouls,
       SUM(Fouls) /  COUNT(*) AS FoulsPerMatch,
       (SUM(YellowCards) + SUM(RedCards)) * 1.0 / NULLIF(SUM(Fouls), 0) AS CardPerFoulRatio,
       COUNT(*) AS Matchs
FROM (
        SELECT HomeTeam AS Team, HF AS Fouls, HY AS YellowCards, HR AS RedCards
        FROM L1

        UNION ALL

        SELECT AwayTeam AS Team, AF AS Fouls, AY AS YellowCards, AR AS RedCards
        FROM L1
)
GROUP BY Team
ORDER BY FoulsPerMatch DESC, CardsPerMatch DESC
"""
result = pd.read_sql(query, conn)
print(result)

           Team  TotalCards  TotalYellowCards  TotalRedCards  CardsPerMatch  \
0          Lens          94                86              8              2   
1        Rennes          69                67              2              2   
2    St Etienne          68                65              3              2   
3        Monaco          61                57              4              1   
4         Brest          69                66              3              2   
5   Montpellier          94                87              7              2   
6      Toulouse          57                54              3              1   
7       Auxerre          71                67              4              2   
8         Lille          77                75              2              2   
9     Marseille          70                64              6              2   
10       Nantes          73                69              4              2   
11        Reims          68                63       

## Possibilité d’upsets (surprises)

- comparer la cote du favori (B365H, B365A) avec le résultat effectif (FTR)

In [27]:
query = """
SELECT HomeTeam, AwayTeam, Favori, FTR AS ResultatsMatchs,
       CASE
           WHEN Favori = 'Home' AND FTR = 'H' THEN 'Win'
           WHEN Favori = 'Away' AND FTR = 'A' THEN 'Win'
           ELSE 'Lose'
       END AS ResultatsParis
FROM (
        SELECT HomeTeam, AwayTeam, FTR,
               CASE
                   WHEN B365H < B365A THEN 'Home'
                   WHEN B365A < B365H THEN 'Away'
                   ELSE 'Draw'
               END AS Favori
        FROM L1
) AS T
"""
result = pd.read_sql(query, conn)
print(result)

       HomeTeam     AwayTeam Favori ResultatsMatchs ResultatsParis
0      Le Havre     Paris SG   Away               A            Win
1         Brest    Marseille   Away               A            Win
2         Reims        Lille   Away               A            Win
3        Monaco   St Etienne   Home               H            Win
4       Auxerre         Nice   Away               H           Lose
..          ...          ...    ...             ...            ...
301      Nantes  Montpellier   Home               H            Win
302        Nice        Brest   Home               H            Win
303    Paris SG      Auxerre   Home               H            Win
304  St Etienne     Toulouse   Home               A           Lose
305  Strasbourg     Le Havre   Home               A           Lose

[306 rows x 5 columns]


## Analyse de la variance des cotes

- comparer MaxH / MinH pour voir les matchs où les bookmakers n’étaient pas d’accord

In [76]:
query = """
SELECT Date, HomeTeam, AwayTeam, FTR, MinH, MaxH, DiffH,
CASE
    WHEN DiffH > 0.5 THEN 'Yes'
    ELSE 'No'
END AS Disagreement
FROM (
        SELECT Date,
               HomeTeam,
               AwayTeam,
               FTR,
               MIN(B365H, BWH, BFH, PSH, WHH, "1XBH") AS MinH,
               MaxH,
               MaxH - MIN(B365H, BWH, BFH, PSH, WHH, "1XBH") AS DiffH
        FROM L1
) AS T
"""
# result = pd.read_sql(query, conn)
# print(result)

cols = ['B365H','BWH','BFH','PSH','WHH','1XBH']

# Calculer MinH en gérant les valeurs NULLS
df["MinH"] = df[cols].min(axis=1, skipna=True)

# Calculer DiffH
df["DiffH"] = df["MaxH"] - df["MinH"]

# Définir le Disagreement
df['Disagreement'] = df['DiffH'].apply(lambda x: 'Yes' if x > 0.5 else 'No')

# Affichage
df_result = df[['Date', 'HomeTeam', 'AwayTeam', 'FTR', 'MinH', 'MaxH', 'DiffH', 'Disagreement']]

print(df_result)


           Date    HomeTeam     AwayTeam FTR  MinH  MaxH  DiffH Disagreement
0    16/08/2024    Le Havre     Paris SG   A  6.75  8.00   1.25          Yes
1    17/08/2024       Brest    Marseille   A  2.80  3.00   0.20           No
2    17/08/2024       Reims        Lille   A  3.30  3.60   0.30           No
3    17/08/2024      Monaco   St Etienne   H  1.48  1.54   0.06           No
4    18/08/2024     Auxerre         Nice   H  3.10  3.28   0.18           No
..          ...         ...          ...  ..   ...   ...    ...          ...
301  17/05/2025      Nantes  Montpellier   H  0.00  1.54   1.54          Yes
302  17/05/2025        Nice        Brest   H  1.46  1.53   0.07           No
303  17/05/2025    Paris SG      Auxerre   H  1.18  1.21   0.03           No
304  17/05/2025  St Etienne     Toulouse   A  1.95  2.70   0.75          Yes
305  17/05/2025  Strasbourg     Le Havre   A  1.57  1.60   0.03           No

[306 rows x 8 columns]


# 3. Analyses "pari sportif"

## Taux de victoire attendu vs réel

- transformer une cote (1 / cote) en probabilité implicite, comparer avec la fréquence réelle des victoires/nuls

In [91]:
query = """
WITH Frequencies AS (
    SELECT
        COUNT(CASE WHEN FTR='H' THEN 1 END) * 1.0 / COUNT(*) AS FreqH,
        COUNT(CASE WHEN FTR='D' THEN 1 END) * 1.0 / COUNT(*) AS FreqD,
        COUNT(CASE WHEN FTR='A' THEN 1 END) * 1.0 / COUNT(*) AS FreqA
    FROM L1
)
SELECT FTR AS Resultat,
       AVG(1.0 / B365H) AS ProbH,
       AVG(1.0 / B365D) AS ProbD,
       AVG(1.0 / B365A) AS ProbA,
       f.FreqH,
       f.FreqD,
       f.FreqA
FROM L1
CROSS JOIN Frequencies f
GROUP BY FTR
ORDER BY Resultat
"""
result = pd.read_sql(query, conn)
print(result)

  Resultat     ProbH     ProbD     ProbA    FreqH     FreqD     FreqA
0        A  0.367512  0.266478  0.420658  0.46732  0.202614  0.330065
1        D  0.440037  0.266147  0.348905  0.46732  0.202614  0.330065
2        H  0.541087  0.248256  0.266130  0.46732  0.202614  0.330065


## Valeur des paris "over/under 2.5 buts"

- comparer la prédiction des bookmakers avec le nombre réel de matchs > 2.5 buts

## Analyse des handicaps asiatiques

- voir si les résultats réels sont cohérents avec les handicaps

## Simulation de stratégie

- appliquer une stratégie simple (ex. toujours parier sur le favori à domicile avec cote < 1.5) et calculer le ROI